In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from tensorflow import keras
keras.__version__

In [ ]:
# Obter o DataSet Fashion Mnist: https://www.kaggle.com/zalando-research/fashionmnist
# Está disponível nos datasets do Keras: https://keras.io/api/datasets/

# Criar arrays NumPy contendo os conjuntos de treino, validação e teste

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] , X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


In [ ]:
y_test.shape

In [ ]:
# Normalizar para o intervalo [0,1] e tranformar para valores reais

X_train = X_train.astype('float32') / 255
X_valid = X_valid.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# As labels têm valores numéricos e este array permite fazer a transformação para o nome da classe

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


In [ ]:
# Exercicio
# Dimensão de cada um dos conjuntos - A que corresponde cada uma das dimensões?
# Qual o formato de input de cada elemento que será apresentado à NN?




In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
save_fig('fashion_mnist_plot', tight_layout=False)
plt.show()

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Utilizar a API Sequencial do Keras para construir o modelo
# https://keras.io/api/

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
# Obter Informação sobre o modelo
model.layers


In [ ]:
# Obter Informação sobre o modelo
model.summary()


In [ ]:
#Analisar detalhadamente uma camada

hidden1 = model.layers[1]
weights, biases = hidden1.get_weights()

print('Layer ', hidden1.name)
print('Weights with shape ', weights.shape, ' :\n', weights)
print('Biases with shape ', biases.shape, ' :\n', biases)

In [ ]:
# Compilar o modelo: Indicar loss, otimizador e métrica a utilizar
# Sparse Categorical Cross Entropy -> Métrica Cross Entropy para classificação com sparse labels, 
# ou seja, para situações em que cada instância tem um label
# Consultar a documentação para verificar quais as situações em que deve ser usada a Categorical Cross Entropy
# ou a Binary Cross Entropy

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=["accuracy"])

In [ ]:
# Treinar o modelo: indicar dados de treino e validação, número de épocas
# Batch Size = 32 (Default)
# Devolve um objeto history contendo os valores obtidos durante o treino

history = model.fit(X_train, y_train, batch_size=10, epochs=5, 
                    validation_data=(X_valid, y_valid))

In [ ]:
# Este dicionário contém as métricas recolhidas durante o treino
history.history.keys()


In [ ]:
# Valores recolhidos da loss de treino
history.history['loss']


In [ ]:
#Gráfico que mostra a evolução das curvas de treino e de validação
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
save_fig("keras_learning_curves_plot")
plt.show()

In [ ]:
# Exercício: Alterar alguns hiper-parâmetros e verificar a alteração de desempenho


In [ ]:
# Avaliação do modelo com o conjunto de teste
model.evaluate(X_test, y_test)


In [ ]:
# Obter previsões para 5 exemplos de teste

init = 0 
X_new = X_test[init:init+5]
y_new = y_test[init:init+5]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
y_pred = np.argmax(model.predict(X_new), axis=-1)
np.array(class_names)[y_pred]


In [ ]:
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 5, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_new[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
save_fig('fashion_mnist_images_plot', tight_layout=False)
plt.show()

In [ ]:
#Gravar o modelo
# O modelo (arquitetura, hiper-parâmetros e parâmetros) podem ser guardados em disco no formato HDF5

model.save("My_First_Model.h5")

# O modelo poderá ser mais tarde recuperado com o método load_model()